In [1]:
pip install selenium

In [2]:
import csv
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 🔹 Paramètres
driver_path = "msedgedriver.exe"
nombre_recommandations = 50
csv_filename = "nouveau_resultats_recommandations.csv"  # Nouveau fichier pour sauvegarde

# 🔹 Liens des vidéos YouTube et le nombre de tests restants pour chaque (30 tests par vidéo)
videos = {
    "Ninho feat. Niska": 30,
    "Kabbsky solo": 30,
    "Sayler solo": 30,
    "Sayler feat. Kabbsky": 30,
}

video_urls = {
    "Ninho feat. Niska": "https://www.youtube.com/watch?v=yGJLE1UXBjo",
    "Kabbsky solo": "https://www.youtube.com/watch?v=uW0BksyesKE",
    "Sayler solo": "https://www.youtube.com/watch?v=jYfPxaP_XN4",
    "Sayler feat. Kabbsky": "https://www.youtube.com/watch?v=aKe0AcYbO8Q",
}

artistes = ["Ninho", "Niska", "Kabbsky", "Sayler"]

with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Video Testée", "Test Numéro", "Artiste", "Nombre d'apparitions"])

print("\n🔹 Début des tests pour analyser la probabilité d'apparition des artistes 🔹\n")

for nom_video, tests_restants in videos.items():
    video_url = video_urls[nom_video]

    print(f"📌 Analyse pour '{nom_video}' ({tests_restants} tests restants)")

    for test in range(1, tests_restants + 1):
        print(f"   🔹 Test {test}/{tests_restants} en cours...", end="\r")

        try:
            service = Service(driver_path)
            options = webdriver.EdgeOptions()
            options.add_argument("--incognito")  # Mode navigation privée
            driver = webdriver.Edge(service=service, options=options)

            driver.delete_all_cookies()

            driver.get(video_url)

            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(3)  

            for _ in range(50):  
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
                time.sleep(1.5)  
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-compact-video-renderer")))
            except:
                print(f"⚠️ Aucune vidéo recommandée trouvée pour '{nom_video}'.")

            video_containers = driver.find_elements(By.CSS_SELECTOR, "ytd-compact-video-renderer")[:nombre_recommandations]

            resultats_test = {artiste: 0 for artiste in artistes}

            for video in video_containers:
                try:
                    titre_element = video.find_element(By.CSS_SELECTOR, "#video-title")
                    titre = titre_element.get_attribute("title").strip().lower()  # Convertir en minuscules

                    for artiste in artistes:
                        if artiste.lower() in titre:  # Vérification insensible à la casse
                            resultats_test[artiste] += 1

                except Exception as e:
                    print(f"⚠️ Erreur lors de la récupération d’un titre : {e}")

            # 🔹 Sauvegarde dans un fichier CSV après chaque test 🔹
            with open(csv_filename, mode="a", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                for artiste, count in resultats_test.items():
                    writer.writerow([nom_video, test, artiste, count])

            driver.quit()

            time.sleep(random.uniform(5, 10))

        except Exception as e:
            print(f"⚠️ Erreur pendant le test {test} pour '{nom_video}' : {e}")

    print(f"\n✅ Tests terminés pour '{nom_video}'.\n")

print("\n📊 Analyse terminée. Résultats sauvegardés dans", csv_filename)



🔹 Début des tests pour analyser la probabilité d'apparition des artistes 🔹

📌 Analyse pour 'Ninho feat. Niska' (30 tests restants)
   🔹 Test 30/30 en cours...
✅ Tests terminés pour 'Ninho feat. Niska'.

📌 Analyse pour 'Kabbsky solo' (30 tests restants)
⚠️ Aucune vidéo recommandée trouvée pour 'Kabbsky solo'.
   🔹 Test 30/30 en cours...
✅ Tests terminés pour 'Kabbsky solo'.

📌 Analyse pour 'Sayler solo' (30 tests restants)
   🔹 Test 30/30 en cours...
✅ Tests terminés pour 'Sayler solo'.

📌 Analyse pour 'Sayler feat. Kabbsky' (30 tests restants)
   🔹 Test 30/30 en cours...
✅ Tests terminés pour 'Sayler feat. Kabbsky'.


📊 Analyse terminée. Résultats sauvegardés dans nouveau_resultats_recommandations.csv
